In [28]:
import os
import asyncio

from prefect.deployments import run_deployment
from prefect.client.orchestration import get_client
from prefect.client.schemas.actions import WorkPoolCreate

from k8s_flow import my_k8s_flow


In [ ]:
WORK_POOL_NAME = "mypool"

# Create pool if it doesn't exist
async with get_client() as client:
    await client.hello()
    pools = await client.read_work_pools()
    pool_exists = any(p.name == WORK_POOL_NAME for p in pools)
    if pool_exists:
        print(f"Work pool '{WORK_POOL_NAME}' already exists")
    else:
        print(f"Creating work pool '{WORK_POOL_NAME}'...")
        await client.create_work_pool(
            work_pool=WorkPoolCreate(name=WORK_POOL_NAME, type="kubernetes")
        )
        print(f"Work pool '{WORK_POOL_NAME}' created")

Work pool 'mypool' already exists


In [ ]:
# --- CONFIGURATION ---
PREFECT_API_URL = "http://localhost:4200/api"
WORK_POOL_NAME = "mypool"
IMAGE_NAME = "prefecthq/prefect:3-python3.12"
DEPLOYMENT_NAME = "k8s-simple-deployment"

os.environ["PREFECT_API_URL"] = PREFECT_API_URL

# --- EXECUTION BLOCK ---
async def deploy_and_run():
    # Import inside function to avoid "interactive" detection
    
    print(f"Connecting to Server: {PREFECT_API_URL}")
    
    # 1. Create Deployment
    deployment_id = await my_k8s_flow.deploy(
        name=DEPLOYMENT_NAME,
        work_pool_name=WORK_POOL_NAME,
        image=IMAGE_NAME
    )
    print(f"Deployment '{DEPLOYMENT_NAME}' created")
    
    # 2. Trigger Run
    print("Triggering run...")
    run = await run_deployment(
        name=f"my-k8s-flow/{DEPLOYMENT_NAME}",
        timeout=0
    )
    print(f"Run triggered! ID: {run.id}")
    return run

await deploy_and_run()

Connecting to Server: http://localhost:4200/api


ValueError: Work pool 'mypool' does not support custom Docker images. Please use a work pool with an `image` variable in its base job template or specify a remote storage location for the flow with `.from_source`. If you are attempting to deploy a flow to a local process work pool, consider using `flow.serve` instead. See the documentation for more information: https://docs.prefect.io/latest/how-to-guides/deployments/run-flows-in-local-processes

In [30]:
# Cell 1: Imports
import os
from prefect.deployments import run_deployment
from prefect.client.orchestration import get_client
from prefect.client.schemas.actions import WorkPoolCreate

os.environ["PREFECT_API_URL"] = "http://localhost:4200/api"

# Cell 2: Create work pool (run once)
async with get_client() as client:
    pools = await client.read_work_pools()
    if not any(p.name == "mypool" for p in pools):
        await client.create_work_pool(
            work_pool=WorkPoolCreate(name="mypool", type="kubernetes")
        )
        print("Work pool created")

# Cell 3: Deploy and run
from k8s_flow import my_k8s_flow

await my_k8s_flow.deploy(
    name="demo",
    work_pool_name="mypool",
    push=False
)
run = await run_deployment(name="my-k8s-flow/demo")
print(f"Run: {run.id}")

ValueError: Either an image or remote storage location must be provided when deploying a deployment.